# IBM Cloud Pak for Data Data Virtualization Lab Setup

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

The IBM Cloud Pak for Data Console is only one way you can interact with the Virtualization service. IBM Cloud Pak for Data is built on a set of microservices that communicate with each other and the Console user interface using RESTful APIs. You can use these services to automate anything you can do throught the user interface.

This Jupyter Notebook contains examples of how to use the Open APIs to retrieve information from the virtualization service, how to run SQL statements directly against the service through REST and how to provide authoritization to objects. This provides a way write your own script to automate the setup and configuration of the virtualization service.

The next part of the lab relies on a set of base classes to help you interact with the RESTful Services API for IBM Cloud Pak for Data Virtualization. You can access this library on GITHUT. The commands below download the library and run them as part of this notebook.
<pre>
&#37;run CPDDVRestClass.ipynb
</pre>
The cell below loads the RESTful Service Classes and methods directly from GITHUB. Note that it will take a few seconds for the extension to load, so you should generally wait until the "Db2 Extensions Loaded" message is displayed in your notebook. 
1. Click the cell below
2. Click **Run**

In [ ]:
!wget -O CPDDVRestClass.ipynb https://raw.githubusercontent.com/Db2-DTE-POC/CPDDVLAB/master/CPDDVRestClass.ipynb
%run CPDDVRestClass.ipynb

## Establishing a Connection to the Console

### Connections
To connect to the Data Virtualization service you need to provide the URL, the service name (v1) and profile the console user name and password. 

In [ ]:
# Connect to the Db2 Data Management Console service

# From Outside the Cluster
# Console  = 'https://services-uscentral.skytap.com:9152'
# From Inside the Cluster
Console  = 'https://openshift-skytap-nfs-lb.ibm.com'
user     = 'admin'
password = 'DTE.cp4data!'

# Set up the required connection
CPDAPI = Db2(Console)
api = '/v1'
CPDAPI.authenticate(api, user, password)
database = Console

## Utility Routines

### Data Sources

In [ ]:
# Display the Available Data Sources already configured
json = CPDAPI.getDataSources()
CPDAPI.displayResults(json) 

### Virtualized Tables and Views

In [ ]:
# Display the Virtualized Assets Avalable to Engineers and Users
roles = ['DV_ENGINEER','DV_USER']
for role in roles:
    r = CPDAPI.getRole(role)
    if (CPDAPI.getStatusCode(r)==200):
        json = CPDAPI.getJSON(r)
        df = pd.DataFrame(json_normalize(json['objects']))
        print(', '.join(list(df)))
        display(df)
    else:
        print(CPDAPI.getStatusCode(r))  

In [ ]:
### Display All Virtualized Tables and Views
display(CPDAPI.getVirtualizedTablesDF())
display(CPDAPI.getVirtualizedViewsDF())

### Cloud Pak for Data User Management

In [ ]:
# Get the list of CPD Users
r = CPDAPI.getUsers()
if (CPDAPI.getStatusCode(r)==200):
    json = CPDAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print(', '.join(list(df)))
    display(df[['uid','username','displayName']])
else:
    print(CPDAPI.getStatusCode(r))

In [ ]:
# Add a Single user to CPD
username = "LABUSER1"
displayName = "LABUSER1"
email = "kohlmann@ca.ibm.com"
user_roles = ["Data Scientist"]
password = 'password'
r = CPDAPI.addUser(username, displayName, email, user_roles, password)
if (CPDAPI.getStatusCode(r)==201):
    print('User Added')
else:
    print(CPDAPI.getStatusCode(r))

## Lab Setup

### Add Users to CPD and Data Virtualization

In [ ]:
# Add x Data Scientists, LABUSERS to CPD

ids = 10
userList = {'UserRoot':['LABUSER','LABDATAENGINEER'],'Role':[['Data Scientist','Developer'],['Data Engineer']]}
userListDF = pd.DataFrame(userList) 
email = 'kohlmann@ca.ibm.com'
password = 'dtedvlab'

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        username = userListDF['UserRoot'].iloc[row]+str(x)
        user_role = userListDF['Role'].iloc[row]
        displayName = username
        r = CPDAPI.addUser(username, displayName, email, user_role, password)
        if (CPDAPI.getStatusCode(r)==201):
            print('User: '+username+' Added as a '+str(user_role))
        else:
            print(CPDAPI.getStatusCode(r))

In [ ]:
# Add x Users and Engineers to the DV Service
ids = 10
userList = {'UserRoot':['LABUSER','LABDATAENGINEER'],'Role':['User','Engineer']}
userListDF = pd.DataFrame(userList) 

df = CPDAPI.getUsersDF() # Get existing list of users to get the uid

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        display_name = userListDF['UserRoot'].iloc[row]+str(x)
        role = userListDF['Role'].iloc[row]
        
        r = CPDAPI.addUserToDV(display_name, role, df)
        if (CPDAPI.getStatusCode(r)==200):
            print('User: '+display_name+' added to Data Virtualization Service')
        else:
            print(CPDAPI.getStatusCode(r))

### Grant Access to Data Engineers to Existing Views and Tables

In [ ]:
# Grant Access to Data Engineers to all the Views owned by the logged in user
ViewsDF = CPDAPI.getVirtualizedViewsDF()
roleToGrant = 'DV_ENGINEER'
for index, row in ViewsDF.iterrows():
    name = row['viewname']
    schema = row['viewschema']

    r = CPDAPI.grantPrivledgeToRole(name, schema, roleToGrant)
    if (CPDAPI.getStatusCode(r)==200):
        print('Access granted')
    else:
        print(CPDAPI.getStatusCode(r))

In [ ]:
# Grant Access to Data Engineers to all the Virtualizated Tables owned by the logged in user
TablesDF = CPDAPI.getVirtualizedTablesDF()
roleToGrant = 'DV_ENGINEER'
for index, row in TablesDF.iterrows():
    name = row['table_name']
    schema = row['table_schema']

    r = CPDAPI.grantPrivledgeToRole(name, schema, roleToGrant)
    if (CPDAPI.getStatusCode(r)==200):
        print('Access granted')
    else:
        print(CPDAPI.getStatusCode(r))

### Test Existing Views

In [ ]:
#Testing Views
sqlText = 'SELECT * FROM TRADING.OHIO;'

print(CPDAPI.runSQL(sqlText))

## Lab Teardown

### Drop the Existing Project

### Remove Tables and Views Created by Lab Participants

In [ ]:
# Delete Virtualized Tables Created by Lab Participants

virtualTables = CPDAPI.getVirtualizedTablesDF()
virtualUserTables = virtualTables.loc[virtualTables['owner'] != 'USER999']
display(virtualUserTables)
for index, row in virtualUserTables.iterrows():
    schema = row['table_schema']
    table = row['table_name']
    source = row['data_source_table_name']
    CPDAPI.deleteVirtualizedTable(schema, table, source)
    if (CPDAPI.getStatusCode(r)==200):
        print('Virtualized Table deleted')
    else:
        print(CPDAPI.getStatusCode(r))
display(CPDAPI.getVirtualizedTablesDF())

In [ ]:
# Delete Virtualized Views Created by Lab Participants

views = CPDAPI.getVirtualizedViewsDF()
userViews = views.loc[views['owner'] != 'USER999']
display(userViews)
for index, row in userViews.iterrows():
    schema = row['viewschema']
    view = row['viewname']
    CPDAPI.deleteView(schema, view)
    if (CPDAPI.getStatusCode(r)==200):
        print('View deleted')
    else:
        print(CPDAPI.getStatusCode(r))
display(CPDAPI.getVirtualizedViewsDF())

### Remove Users from Data Virtualization Server and CPD

In [ ]:
# Drop x users and engineers from the DV Service
ids = 10
userList = {'UserRoot':['LABUSER','LABDATAENGINEER']}
userListDF = pd.DataFrame(userList) 

df = CPDAPI.getUsersDF() # Get existing list of users to get the uid

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        display_name = userListDF['UserRoot'].iloc[row]+str(x)
        
        r = CPDAPI.dropUserFromDV(display_name, df)
        if (CPDAPI.getStatusCode(r)==200):
            print('User: '+display_name+' dropped from Data Virtualization Service')
        else:
            print(CPDAPI.getStatusCode(r))

In [ ]:
# Drop x users and engineers from CPD
ids = 10
userList = {'UserRoot':['labuser','labdataengineer']}
userListDF = pd.DataFrame(userList) 

for x in range(0, ids):
    for row in range(0, len(userListDF)):
        username = userListDF['UserRoot'].iloc[row]+str(x)

        r = CPDAPI.dropUser(username)
        if (CPDAPI.getStatusCode(r)==200):
            print('User: '+username+' Dropped')
        else:
            print(CPDAPI.getStatusCode(r))

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]